In [1]:
import sys
sys.path.insert(0, "..")
import json

from src.db_ingestion.chroma_client import query_to_collection
from src.talent_selection_flow.crews.cv_to_job_crew.crew import CVToJobCrew

from src.db_ingestion.metadata_extraction_crew.crews import CVMetadataExtractorCrew
from src.db_ingestion.metadata_extraction_crew.enums import ExperienceLevel, EducationLevel

In [2]:
cv_example = "Experienced Software Engineer with 5+ years in Python, specialized in building scalable AI agents and distributed systems using CrewAI and FastAPI. From Houston"
top_k = 3

In [11]:
# Extract cv metadata
metadata = CVMetadataExtractorCrew(
    guardrail_max_retries=3,
    verbose=False,
    human_input=False,
).crew().kickoff(inputs={
    "content": cv_example,
    "educationlevel_options": "/".join(EducationLevel),
    "experiencelevel_options": "/".join(ExperienceLevel),
})
metadata_dict = json.loads(metadata.raw)

# Get matches from jobs collection
related_jobs = query_to_collection(
    collection_name="jobs",
    query_text=cv_example,
    country=metadata_dict.get("country"),
    top_k=3,
)

# Send info to CVToJobCrew
cv_crew = CVToJobCrew(
    verbose=False,
    guardrail_max_retries=3,
)
result = cv_crew.crew().kickoff(
    inputs={"structured_cv": metadata_dict,
            "related_jobs": related_jobs}
)

2026-02-16 13:28:33.229 | DEBUG    | src.db_ingestion.metadata_extraction_crew.guardrails:validate_cvmetadata_schema:12 - Guardrail input:
{"skills": "Python, CrewAI, FastAPI", "industries": "AI, Distributed Systems", "experience_level": "senior", "country": "US", "summary": "Experienced Software Engineer with 5+ years in Python, specialized in building scalable AI agents and distributed systems using CrewAI and FastAPI.", "education_level": "unknown", "languages": "unknown"}
2026-02-16 13:28:33.654 | INFO     | src.db_ingestion.chroma_client:query_to_collection:153 - Initiating vector search in collection 'jobs' (Top K: 3)
2026-02-16 13:28:34.250 | DEBUG    | src.db_ingestion.chroma_client:query_to_collection:175 - Final formatted results:
{'2417': {'title': 'Sr. Developer - Python/Java', 'similarity': 0.8087, 'skills': 'Python, Oracle, Teradata, Linux, Java, shell scripting, Integration, XML, XLST, WCF, SOAP, JSON, RESTful services, systems, application frameworks, database optimizat